In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint

import pandas as pd
import numpy as np

import json

In [2]:
class DataPipeline_sj:
    
    def find_vac(self, vac):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, /'
                            'like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        
        main_link = 'https://russia.superjob.ru'
        
        vacancies = []
     
        params = {'keywords':vac, 'page':''}

        html = requests.get(main_link + '//vacancy/search/', headers=headers, params=params).text
        soup = bs(html, 'html.parser')

        pager_block = soup.find('div', {'class': '_3zucV L1p51 undefined _2guZ- _GJem'})
        page_n = int(pager_block.find_all('span', {'class': '_3IDf-'})[-2].getText())

        for page in range(1, page_n):
            params = {'keywords': vac, 'page': page}
            html = requests.get(main_link + '//vacancy/search/', headers=headers, params=params).text
            soup = bs(html, 'html.parser')

            vacancy_list = soup.find_all('div', {'class': 'Fo44F QiY08 LvoDO'})

            for vacancy in vacancy_list:
                vacancy_data = {}
                vacancy_name = vacancy.find('a', {'class': 'icMQ_'}).getText()
                vacancy_link = main_link + vacancy.find('a', {'class': 'icMQ_'})['href']
                v_salary = vacancy.find('span',
                                        {'class': 'f-test-text-company-item-salary'}).getText()

                v_salary = v_salary.split()

                if 'По' in v_salary:
                    currency = None
                    salary_min = None
                    salary_max = None
                elif 'от' in v_salary:
                    currency = v_salary[-1]
                    salary_min = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                    salary_max = None
                elif 'до' in v_salary:
                    currency = v_salary[-1]
                    salary_min = None
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                elif '—' in v_salary:
                    currency = v_salary[-1]
                    v_sal = ''.join(v_salary).split('—')
                    salary_min = int(v_sal[0])
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_sal[1]))))
                else:
                    currency = v_salary[-1]
                    salary_min = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_salary))))


                vacancy_data['name'] = vacancy_name
                vacancy_data['link'] = vacancy_link
                vacancy_data['salary_min'] = salary_min
                vacancy_data['salary_max'] = salary_max
                vacancy_data['currency'] = currency
                vacancy_data['site'] = main_link
                
                vacancies.append(vacancy_data)
                last_page = soup.find('a', {'class': 'f-test-button-dalshe'})
            
            if not last_page:
                break

        return vacancies

In [3]:
vac_data_js = DataPipeline_sj()

In [4]:
vac_data_js = vac_data_js.find_vac('data')
pprint(vac_data_js)

[{'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/data-scientist-33826570.html',
  'name': 'Data scientist, специалист по обучению нейронных сетей',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://russia.superjob.ru'},
 {'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/genetik-analitik-dannyh-33864238.html',
  'name': 'Генетик-аналитик данных / Data scientist',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://russia.superjob.ru'},
 {'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/data-scientist-33832075.html',
  'name': 'Data scientist',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://russia.superjob.ru'},
 {'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/java-programmist-33790973.html',
  'name': 'Java программист (Spring)',
  'salary_max': None,
  'salary_min': 100000,
  'site': 'https://russia.superjob.ru'},
 {'currency': None,
  'link': 'https://russia.superjob.ru/vakans

In [5]:
with open("vac_sj_up.json", "w") as write_file:
    json.dump(vac_data_js, write_file)